## Build a News ETL Data Pipeline Using Python and SQLite

### Task 1: Import Libraries and Connect to the News API

In [ ]:
import pandas as pd
import sqlite3
import logging
from newsapi import NewsApiClient

In [ ]:
news_api_key = "69651cc0cfc54c9087b1c98867e8353c"  
news_api = NewsApiClient(api_key=news_api_key)

### Task 2: Retrieve and Print News Articles

In [ ]:
def extract_news_data():
    try:
        result = news_api.get_everything(q="AI", language="en",sort_by='publishedAt')
        logging.info("Connection is successful.")
        return result["articles"]
    except:
        logging.error("Connection is unsuccessful.")
        return None

articles = extract_news_data()

print(articles[:3])

### Task 3: Clean Author Column

In [ ]:
def clean_author_column(text):
    try:
        return text.split(",")[0].title()
    except AttributeError:
        return "No Author"

### Task 4: Transform News Data

In [ ]:
def transform_news_data(articles):
    article_list = []
    for i in articles:
        article_list.append([value.get("name", 0) if key == "source" else value for key, value in i.items() if key in ["author", "title", "publishedAt", "content", "url", "source"]])

    df = pd.DataFrame(article_list, columns=["Source", "Author Name", "News Title", "URL", "Date Published", "Content"])

    df["Date Published"] = pd.to_datetime(df["Date Published"]).dt.strftime('%Y-%m-%d %H:%M:%S')
    df["Author Name"] = df["Author Name"].apply(clean_author_column)
 
    return df

transformed_data = transform_news_data(articles)

print(transformed_data)

### Task 5: Load the Data into SQLite Database


In [ ]:
def load_news_data(data):
    with sqlite3.connect("/usercode/news_data.sqlite") as connection:
        cursor = connection.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS news_table (
                "Source" VARCHAR(30),
                "Author Name" TEXT,
                "News Title" TEXT,
                "URL" TEXT,
                "Date Published" TEXT,
                "Content" TEXT
            )
        ''')
    data.to_sql(name="news_table", con=connection, index=False, if_exists="append")
 
load_news_data(transformed_data)

### Task 8: Verify Data Loading

In [ ]:
with sqlite3.connect("/usercode/etl_news_data.sqlite") as connection:
    df = pd.read_sql("SELECT * FROM news_table;", connection)
df.head()